In [42]:
import scipy.sparse
import pickle

import networkx as nx
import numpy as np
import pandas as pd

from sklearn.decomposition import TruncatedSVD

In [43]:
'''
Retrieve the array obtained by apllying the dimentinality reductin algorithm
graph_matrix: SHAPE: (channels, n_comp)
'''
n_comp = 250
graph_matrix = np.load('../../../dlabdata1/youtube_large/jouven/graph_matrix_reduced_'+str(n_comp)+'.npz')
graph_matrix = graph_matrix['arr_0']
df = pd.DataFrame(graph_matrix)

In [44]:
df = df.rename(lambda x: 'dr'+str(x), axis='columns')

In [45]:
df.head()

,dr0,dr1,dr2,dr3,dr4,dr5,dr6,dr7,dr8,dr9,...,dr240,dr241,dr242,dr243,dr244,dr245,dr246,dr247,dr248,dr249
0,3.018157,2.509277,-1.370723,-1.205678,-2.077085,-0.748989,0.437611,2.537465,-1.731717,-0.114103,...,0.462441,0.120393,0.235863,0.140165,0.190139,-0.138168,-0.537050,0.449095,0.367694,-0.142471
1,8.037677,4.613102,1.003205,0.260758,2.158020,-4.919136,1.621984,0.779602,2.237350,2.801618,...,-0.973995,-0.326946,0.129347,1.063669,1.069335,0.236451,0.575651,-0.887077,0.703395,-0.172182
2,21.643678,4.486470,17.017157,4.962999,5.291304,-7.874998,-0.042691,5.202206,5.295214,3.251302,...,-0.966028,-2.156922,2.965868,2.623973,2.167879,2.348487,0.090519,0.440349,-7.247206,-2.688719
3,367.470984,-22.194127,-71.681107,-28.139755,-121.998349,-263.544879,40.496879,-23.376365,14.612321,31.488560,...,-3.349631,-1.126843,7.156940,-6.779123,-0.874278,5.733782,6.967574,23.164244,-7.655783,6.040870
4,27.062345,3.521666,8.440126,2.053278,10.615795,-12.821902,-3.522644,-7.847089,-14.845779,20.498252,...,-3.109735,-0.237170,4.575755,-0.552023,4.367416,2.745388,-3.128423,0.606416,2.628945,-1.766331


In [46]:
df = df.reset_index()
df = df.rename(columns = {'index': 'source'})
df.head()

,source,dr0,dr1,dr2,dr3,dr4,dr5,dr6,dr7,dr8,...,dr240,dr241,dr242,dr243,dr244,dr245,dr246,dr247,dr248,dr249
0,0,3.018157,2.509277,-1.370723,-1.205678,-2.077085,-0.748989,0.437611,2.537465,-1.731717,...,0.462441,0.120393,0.235863,0.140165,0.190139,-0.138168,-0.537050,0.449095,0.367694,-0.142471
1,1,8.037677,4.613102,1.003205,0.260758,2.158020,-4.919136,1.621984,0.779602,2.237350,...,-0.973995,-0.326946,0.129347,1.063669,1.069335,0.236451,0.575651,-0.887077,0.703395,-0.172182
2,2,21.643678,4.486470,17.017157,4.962999,5.291304,-7.874998,-0.042691,5.202206,5.295214,...,-0.966028,-2.156922,2.965868,2.623973,2.167879,2.348487,0.090519,0.440349,-7.247206,-2.688719
3,3,367.470984,-22.194127,-71.681107,-28.139755,-121.998349,-263.544879,40.496879,-23.376365,14.612321,...,-3.349631,-1.126843,7.156940,-6.779123,-0.874278,5.733782,6.967574,23.164244,-7.655783,6.040870
4,4,27.062345,3.521666,8.440126,2.053278,10.615795,-12.821902,-3.522644,-7.847089,-14.845779,...,-3.109735,-0.237170,4.575755,-0.552023,4.367416,2.745388,-3.128423,0.606416,2.628945,-1.766331


In [47]:
'''
Need to prepare the dataframe before transforming the dataframe into a networkx graph.
we need source and dest columns corresponding the edge and a weight column.
'''

df_prepared_for_graph = df.melt(id_vars=["source"], 
        var_name="dest", 
        value_name="weight")

In [48]:
df_prepared_for_graph.head()

,source,dest,weight
0,0,dr0,3.018157
1,1,dr0,8.037677
2,2,dr0,21.643678
3,3,dr0,367.470984
4,4,dr0,27.062345


In [49]:
# Channels that are in set_crawler dataset and also in which the language is in english
with open('../../../dlabdata1/youtube_large/olam/channels_id.pickle', 'rb') as f:
    channels_id = pickle.load(f)
f.close()

In [50]:
# Dictionnary to map the channel id to an integer corresponding to the column/row of the sparse matrix.
channel_index_dict = {}
for ind, channel_id in enumerate(channels_id):
    channel_index_dict[ind] = channel_id

In [51]:
channel_index_dict

{0: 'UC44G0QdnNQbBLkvK5HybamQ',
 1: 'UC1CK0ojo3qBdpwqZ7h8pzmw',
 2: 'UCEC2MYEhVliKyHYAl8LKTIQ',
 3: 'UCfRNJiafEm1LBBGFTTq4cXw',
 4: 'UCX2s87V7ZWwl57g9Z8i4nog',
 5: 'UCb1s1OwEvvd_1v4AuKvUh4Q',
 6: 'UCeyNgyN50AYpryjAIkuc-hg',
 7: 'UCz75RVbH8q2jdBJ4SnwuZZQ',
 8: 'UC-weErbn9DNJ4u61d9YIJ0w',
 9: 'UCCRlVNsRLN5rtmo8w5XRV4A',
 10: 'UCF-D_YFujVRQsTRB4GtuQqw',
 11: 'UClyc0EJQe50b1QjvllYYs0w',
 12: 'UCy21T5twk9gYh2oY3CqOtjQ',
 13: 'UCFt7hnmijQ6TrYGuwKAqIig',
 14: 'UCtmncr6bzvuUgYEoSKI52WQ',
 15: 'UClV0PrK-7EMRJaj8hxcujfw',
 16: 'UCoynlb1kbilj6b58ArmuAtw',
 17: 'UCJBayMoDU91Rik6fI0Ao9Vw',
 18: 'UCWQuQhXSfmjEoYoYBtcsFEw',
 19: 'UCEPeWPjaF1CWCKXa3xTp7vw',
 20: 'UCgzWiRT3thPJBu-veBa0y3A',
 21: 'UCt3B6rUXb__X2eMyY7jzgIg',
 22: 'UC-e-HB2xuvc9Su_v07zhEFQ',
 23: 'UCYsQwJsAz-fp4og2Tz3GJrQ',
 24: 'UCcJceGUaevGlP7s2xzL9akA',
 25: 'UC5rD-xsRXA6BQBANuOEmgXA',
 26: 'UC1lXzO1Ss_1w5hS3YwSxLyQ',
 27: 'UCa5e8PYIfV8H_q64-iGGacw',
 28: 'UCHZgztjCu-DhdJcq-ZZLXnA',
 29: 'UCMbt3tLdVM_sfDp6yjXxGoQ',
 30: 'UCPmYONHmw6HNk

In [52]:
df_prepared_for_graph['source'] = df_prepared_for_graph['source'].map(channel_index_dict)

In [53]:
df_prepared_for_graph.head()

,source,dest,weight
0,UC44G0QdnNQbBLkvK5HybamQ,dr0,3.018157
1,UC1CK0ojo3qBdpwqZ7h8pzmw,dr0,8.037677
2,UCEC2MYEhVliKyHYAl8LKTIQ,dr0,21.643678
3,UCfRNJiafEm1LBBGFTTq4cXw,dr0,367.470984
4,UCX2s87V7ZWwl57g9Z8i4nog,dr0,27.062345


In [54]:
graph = nx.from_pandas_edgelist(df_prepared_for_graph, 'source', 'dest', ['weight'], create_using = nx.DiGraph())

In [9]:
graph[0]['dr0']['weight']

3.0181574379196223